<a href="https://colab.research.google.com/github/anash18/4105-HW-Nash/blob/main/HW6/ECGR_4105_HW6_Problem_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from collections import OrderedDict

In [3]:
#Calling the dataset
file_path = r"C:\Users\aaron\Documents\Python Code\Datasets\Housing.csv"
housing = pd.read_csv(file_path)
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [4]:
#Applying a binary map
#mapping the variables to a binary output
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Defining the map function
def mapping(x):
    return x.map({'yes': 1, 'no': 0, 'furnished':  1, 'semi-furnished':  0, 'unfurnished':  -1})

housing[varlist] = housing[varlist].apply(mapping)

housing.head()
#housing.dtypes

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,1
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,1
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,0
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,1
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,1


In [5]:
#Splitting the Dataset
x1_train, x1_val = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state = 0)
x1_train.head()

#Splitting Variables
y1_train = x1_train.pop("price")
y1_val = x1_val.pop("price")

y1_train = y1_train.values.reshape(-1, 1)
y1_val = y1_val.values.reshape(-1, 1)
#print(x1_train)
#print(y1_train)
#x1_train.head()
#print(y1_val)

In [6]:
#Standardizing the Dataset

standard = StandardScaler()

vars1 = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating',
         'airconditioning', 'prefarea', 'furnishingstatus']

x1_train_stan = x1_train
x1_train_stan[vars1] = standard.fit_transform(x1_train_stan[vars1])

x1_val_stan = x1_val
x1_val_stan[vars1] = standard.fit_transform(x1_val_stan[vars1])
#x1_val_stan.head()

In [7]:
#Stacking them there inputs
x1_train_stan_numpy = np.c_[np.ones((len(y1_train), 1)), x1_train_stan[vars1]]
x1_val_stan_numpy = np.c_[np.ones((len(y1_val), 1)), x1_val_stan[vars1]]

#I am become tensor destroyer of deep learning
x1_train_ten = torch.tensor(x1_train_stan_numpy, dtype=torch.float)
x1_val_ten = torch.tensor(x1_val_stan_numpy, dtype=torch.float)
y1_train_ten = torch.tensor(y1_train, dtype=torch.float)
y1_val_ten = torch.tensor(y1_val, dtype=torch.float)

print(x1_train_ten.shape)
print(x1_val_ten.shape)
print(y1_train_ten.shape)
print(y1_val_ten.shape)

torch.Size([436, 13])
torch.Size([109, 13])
torch.Size([436, 1])
torch.Size([109, 1])


In [8]:
#One singular hidden layer it has no friends :(

seq_model_single = nn.Sequential(
            nn.Linear(13, 32), # <1>
            nn.Tanh(),
            nn.Linear(32, 1)) # <2>
seq_model_single

Sequential(
  (0): Linear(in_features=13, out_features=32, bias=True)
  (1): Tanh()
  (2): Linear(in_features=32, out_features=1, bias=True)
)

In [9]:
[param.shape for param in seq_model_single.parameters()]

[torch.Size([32, 13]), torch.Size([32]), torch.Size([1, 32]), torch.Size([1])]

In [10]:
for name, param in seq_model_single.named_parameters():
    print(name, param.shape)

0.weight torch.Size([32, 13])
0.bias torch.Size([32])
2.weight torch.Size([1, 32])
2.bias torch.Size([1])


In [11]:
#Let's define our training loop that will be used

def training_loop(n_epochs, optimizer, model, loss_fn, x_train, x_val,
                  y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        p_train = model(x_train) # <1>
        loss_train = loss_fn(p_train, y_train,)

        p_val = model(x_val) # <1>
        loss_val = loss_fn(p_val, y_val)

        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch: {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}")

In [12]:
#Let's try doing some training

optimizer_1 = optim.SGD(seq_model_single.parameters(), lr=1e-2)

training_loop(
    n_epochs = 5000,
    optimizer=optimizer_1,
    model = seq_model_single,
    loss_fn = nn.MSELoss(),
    x_train = x1_train_ten,
    x_val = x1_val_ten,
    y_train = y1_train_ten,
    y_val = y1_val_ten
)

Epoch: 1, Training loss: 26469982011392.0000, Validation loss: 25189244993536.0000
Epoch: 1000, Training loss: 3545950060544.0000, Validation loss: 2889591291904.0000
Epoch: 2000, Training loss: 3538769412096.0000, Validation loss: 2890860855296.0000
Epoch: 3000, Training loss: 3536646569984.0000, Validation loss: 2891338219520.0000
Epoch: 4000, Training loss: 3535590916096.0000, Validation loss: 2891558420480.0000
Epoch: 5000, Training loss: 3535000567808.0000, Validation loss: 2891685560320.0000


In [13]:
#LIFE OF THE PARTY COMING IN WITH NOT 1, NOT 2, BUT 3 HIDDEN LAYERS

seq_model_multiple = nn.Sequential(
            nn.Linear(13, 32), # <1>
            nn.Tanh(),
            nn.Linear(32, 64), # <2>
            nn.Tanh(),
            nn.Linear(64, 16), # <3>
            nn.Tanh(),
            nn.Linear(16,1))
seq_model_multiple

Sequential(
  (0): Linear(in_features=13, out_features=32, bias=True)
  (1): Tanh()
  (2): Linear(in_features=32, out_features=64, bias=True)
  (3): Tanh()
  (4): Linear(in_features=64, out_features=16, bias=True)
  (5): Tanh()
  (6): Linear(in_features=16, out_features=1, bias=True)
)

In [14]:
[param.shape for param in seq_model_multiple.parameters()]

[torch.Size([32, 13]),
 torch.Size([32]),
 torch.Size([64, 32]),
 torch.Size([64]),
 torch.Size([16, 64]),
 torch.Size([16]),
 torch.Size([1, 16]),
 torch.Size([1])]

In [15]:
for name, param in seq_model_multiple.named_parameters():
    print(name, param.shape)

0.weight torch.Size([32, 13])
0.bias torch.Size([32])
2.weight torch.Size([64, 32])
2.bias torch.Size([64])
4.weight torch.Size([16, 64])
4.bias torch.Size([16])
6.weight torch.Size([1, 16])
6.bias torch.Size([1])


In [16]:
#Let's try doing some training w/friends this time

optimizer_2 = optim.SGD(seq_model_multiple.parameters(), lr=1e-2)

training_loop(
    n_epochs = 5000,
    optimizer=optimizer_2,
    model = seq_model_multiple,
    loss_fn = nn.MSELoss(),
    x_train = x1_train_ten,
    x_val = x1_val_ten,
    y_train = y1_train_ten,
    y_val = y1_val_ten
)

Epoch: 1, Training loss: 26469977817088.0000, Validation loss: 25189242896384.0000
Epoch: 1000, Training loss: 3477441871872.0000, Validation loss: 2559888064512.0000
Epoch: 2000, Training loss: 3477110259712.0000, Validation loss: 2559891734528.0000
Epoch: 3000, Training loss: 3476996751360.0000, Validation loss: 2559895404544.0000
Epoch: 4000, Training loss: 3476958216192.0000, Validation loss: 2559897501696.0000
Epoch: 5000, Training loss: 3476944846848.0000, Validation loss: 2559898812416.0000
